In [52]:
import torch
from timm import create_model
import numpy as np
import cv2
import os
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt
from numpy import linalg as LA

In [4]:
model = create_model('efficientnet_b3',pretrained=True).float()

In [5]:
dir = 'image_normalized'
np_dir = 'image_features'

In [16]:
ap = torch.nn.AdaptiveAvgPool2d((1,1))
for f in tqdm(os.listdir(dir)):
    img = cv2.imread(os.path.join(dir,f))
    imgnorm = cv2.cvtColor(img,cv2.COLOR_BGR2RGB).astype(np.float32)/255.0
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = (imgnorm-mean)/std
    imgtensor = torch.tensor(np.transpose(img,(2,0,1))[np.newaxis,:,:,:]).float()
    with torch.no_grad():
        feature = model.forward_features(imgtensor)
        feature = ap(feature)
        featn = feature.detach().cpu().numpy()
        featn = np.squeeze(featn)
        np.save(os.path.join(np_dir,f.split('.')[0]+'.npy'),featn)

100%|██████████| 24358/24358 [1:53:56<00:00,  3.56it/s]  


In [42]:
image = cv2.imread('still_nox.png')

In [23]:
def subimage(image, rect):
    theta = rect[2]-90
    center = (int(rect[0][0]),int(rect[0][1]))
    height = int(rect[1][0])
    width = int(rect[1][1])
    theta *= 3.14159 / 180 # convert to rad
    v_x = (np.cos(theta), np.sin(theta))
    v_y = (-np.sin(theta), np.cos(theta))
    s_x = center[0] - v_x[0] * ((width-1) / 2) - v_y[0] * ((height-1) / 2)
    s_y = center[1] - v_x[1] * ((width-1) / 2) - v_y[1] * ((height-1) / 2)

    mapping = np.array([[v_x[0],v_y[0], s_x],
                        [v_x[1],v_y[1], s_y]])

    return cv2.warpAffine(image,mapping,(width, height),flags=cv2.WARP_INVERSE_MAP,borderMode=cv2.BORDER_REPLICATE)

In [39]:
imgray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
ret, th = cv2.threshold(imgray, 1,255,cv2.THRESH_BINARY)
contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

In [40]:
len(contours)

2

In [43]:
pills = []
for cont in contours:
    rect = cv2.minAreaRect(cont)
    crop = subimage(image,rect)
    if crop.shape[0] > crop.shape[1]:
            crop = cv2.rotate(crop, cv2.ROTATE_90_CLOCKWISE)
    crop = cv2.resize(crop,(256,int(256*crop.shape[0]/crop.shape[1])))
    pills.append(crop)
normalized = np.zeros((512,512,3),dtype=np.uint8)
normalized[128-pills[0].shape[0]//2 : 128 +(pills[0].shape[0]-pills[0].shape[0]//2),128-pills[0].shape[1]//2 : 128 +(pills[0].shape[1]-pills[0].shape[1]//2)] = pills[0]
normalized[384-pills[1].shape[0]//2 : 384 +(pills[1].shape[0]-pills[1].shape[0]//2),384-pills[1].shape[1]//2 : 384 +(pills[1].shape[1]-pills[1].shape[1]//2)] = pills[1]
cv2.imwrite(os.path.join('stellnox_preprocess.png'),normalized)

True

In [46]:
img = cv2.imread('stellnox_preprocess.png')
imgnorm = cv2.cvtColor(img,cv2.COLOR_BGR2RGB).astype(np.float32)/255.0
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
img = (imgnorm-mean)/std
imgtensor = torch.tensor(np.transpose(img,(2,0,1))[np.newaxis,:,:,:]).float()
with torch.no_grad():
    feature = model.forward_features(imgtensor)
    feature = ap(feature)
    featn = feature.detach().cpu().numpy()
    nox_feat = np.squeeze(featn)

In [53]:
l = []
for feat in tqdm(os.listdir('image_features')):
    feature = np.load(os.path.join('image_features',feat))
    l.append((np.dot(nox_feat,feature)/(LA.norm(nox_feat)*LA.norm(feature)),feat.split('.')[0]))

100%|██████████| 24358/24358 [00:07<00:00, 3096.60it/s]


In [ ]:
l.sort(reverse= True)
l[:100]